In [2]:
# imports liberaries
import warnings 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import re
import json 
import nltk
import spacy
import string
import unicodedata
from bs4 import BeautifulSoup
from textblob import TextBlob 
from nltk.stem import WordNetLemmatizer

from IPython import display 
display.set_matplotlib_formats('svg')
warnings.filterwarnings('ignore')

C:\Users\Mate Kovacs\AppData\Local\Temp\ipykernel_3792\1164050662.py:18: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  display.set_matplotlib_formats('svg')


In [3]:
#load the dataset
data = pd.read_csv("F:\Be.code\sentiment_analysis_nlp\sentiment_analysis_nlp\data\Reviews.csv")
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [10]:
#check the shape
data.shape

(568454, 2)

In [6]:
data = data[['Score','Text']]
data.head()

,Score,Text
0,5,I have bought several of the Vitality canned d...
1,1,Product arrived labeled as Jumbo Salted Peanut...
2,4,This is a confection that has been around a fe...
3,2,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...


In [11]:
# check whether it's having null values or not
data.isnull().sum

<bound method NDFrame._add_numeric_operations.<locals>.sum of         Score   Text
0       False  False
1       False  False
2       False  False
3       False  False
4       False  False
...       ...    ...
568449  False  False
568450  False  False
568451  False  False
568452  False  False
568453  False  False

[568454 rows x 2 columns]>

In [12]:
# check unique categories
data['Score'].value_counts()

5    363122
4     80655
1     52268
3     42640
2     29769
Name: Score, dtype: int64

In [13]:
# if rating is above 3 we will consider it as 1 else as 0.
data['Score'] = data['Score'].apply(lambda x: 1 if x > 3 else 0) # positive as 1 and negative as 0

#### visualiza words using WordCloud

In [14]:
data['Score'].value_counts()

1    443777
0    124677
Name: Score, dtype: int64

In [17]:
from wordcloud import WordCloud, ImageColorGenerator
from PIL import Image

# get the text based on scores!
score_zero = data['Text'][data['Score'] == 0]
score_one = data['Text'][data['Score'] == 1]

#word tokenize the data(convert to string)
def list_tokenizer(Score):
    Scores = " ".join(Score)
    Scores = nltk.word_tokenize(Scores)
    return str(Scores)

# visulizer
def word_cloud(Score, number):
    wc = WordCloud(background_color= 'pink', max_font_size= 45, max_words= 10)
    wc.generate(Score)
    plt.imshow(wc, interpolation= 'bilinear')
    plt.title(f'WorldCloud for {number}')
    plt.axis('off')



In [ ]:
# Let's visualizing the Scores
#word_cloud(list_tokenizer(score_zero), 0)

In [ ]:
#word_cloud(list_tokenizer(score_one), 0)

#### Preprocessing

In [7]:
# 1. lowering words
data['Text'] = data['Text'].str.lower()
data.head()

,Score,Text
0,5,i have bought several of the vitality canned d...
1,1,product arrived labeled as jumbo salted peanut...
2,4,this is a confection that has been around a fe...
3,2,if you are looking for the secret ingredient i...
4,5,great taffy at a great price. there was a wid...


In [8]:
# 2. remove punctuation!
data['Text'] = data['Text'].apply(lambda x: re.sub('[^a-z A-Z 0-9-]+', '', x))
data.head()

,Score,Text
0,5,i have bought several of the vitality canned d...
1,1,product arrived labeled as jumbo salted peanut...
2,4,this is a confection that has been around a fe...
3,2,if you are looking for the secret ingredient i...
4,5,great taffy at a great price there was a wide...


In [9]:
#3. remove stopwords
from spacy.lang.en.stop_words import STOP_WORDS
data['Text'] = data['Text'].apply(lambda x: " ".join([i for i in x.split() if i not in STOP_WORDS]))
data.head()

,Score,Text
0,5,bought vitality canned dog food products found...
1,1,product arrived labeled jumbo salted peanutsth...
2,4,confection centuries light pillowy citrus gela...
3,2,looking secret ingredient robitussin believe f...
4,5,great taffy great price wide assortment yummy ...


In [10]:
#4. remove url and tags! 
data['Text'] = data['Text'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
data.head()

,Score,Text
0,5,bought vitality canned dog food products found...
1,1,product arrived labeled jumbo salted peanutsth...
2,4,confection centuries light pillowy citrus gela...
3,2,looking secret ingredient robitussin believe f...
4,5,great taffy great price wide assortment yummy ...


In [11]:
# 5. remove html tags! 
data['Text'] = data['Text'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

data.head()

,Score,Text
0,5,bought vitality canned dog food products found...
1,1,product arrived labeled jumbo salted peanutsth...
2,4,confection centuries light pillowy citrus gela...
3,2,looking secret ingredient robitussin believe f...
4,5,great taffy great price wide assortment yummy ...


In [ ]:
# 6. spelling correction 
# %time 
#data['Text'] = data['Text'].apply(lambda x: TextBlob(x).correct())

#data.head()

In [12]:
# 7. remove emails
data['Text'] = data['Text'].apply(lambda x: re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+\b)', '', x))
data.head()

,Score,Text
0,5,bought vitality canned dog food products found...
1,1,product arrived labeled jumbo salted peanutsth...
2,4,confection centuries light pillowy citrus gela...
3,2,looking secret ingredient robitussin believe f...
4,5,great taffy great price wide assortment yummy ...


In [13]:
# 8. remove extra spaces
data['Text'] = data['Text'].apply(lambda x: " ".join(x.split()))
data.head()

,Score,Text
0,5,bought vitality canned dog food products found...
1,1,product arrived labeled jumbo salted peanutsth...
2,4,confection centuries light pillowy citrus gela...
3,2,looking secret ingredient robitussin believe f...
4,5,great taffy great price wide assortment yummy ...


In [14]:
# 9. lemmatizer 
%time
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

data["Text"] = data["Text"].apply(lambda text: lemmatize_words(text))
data.head()

CPU times: total: 0 ns
Wall time: 0 ns


,Score,Text
0,5,bought vitality canned dog food product found ...
1,1,product arrived labeled jumbo salted peanutsth...
2,4,confection century light pillowy citrus gelati...
3,2,looking secret ingredient robitussin believe f...
4,5,great taffy great price wide assortment yummy ...


#### Convert text to numbers

In [15]:
# data split 
from sklearn.model_selection import train_test_split 
xtrain, xtest, ytrain, ytest = train_test_split(data['Text'], data['Score'], test_size = 0.3)

In [25]:
xtrain.shape
#xtest.shape

(397917,)

In [16]:
# method 1: Bag of words(BOW)
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

xtrain_bow = vectorizer.fit_transform(xtrain).toarray()
xtest_bow = vectorizer.transform(xtest).toarray()


MemoryError: Unable to allocate 487. GiB for an array with shape (397917, 164319) and data type int64

In [17]:
# method 2: TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer  

tf_vectorizer = TfidfVectorizer()

# let's convert 
xtrain_tf = tf_vectorizer.fit_transform(xtrain).toarray()
xtest_tf = tf_vectorizer.transform(xtest).toarray()

MemoryError: Unable to allocate 487. GiB for an array with shape (397917, 164319) and data type float64

#### model: Gaussina NB


In [26]:
from sklearn.naive_bayes import GaussianNB

#BOW
clf_bow = GaussianNB().fit(xtrain_bow, ytrain)  # fitting 
prediction_bow = clf_bow.predict(xtest_bow)  # predictions

#TF-IDF
clf_tf = GaussianNB().fit(xtrain_tf, ytrain)
prediction_tf = clf_tf.predict(xtest_tf)

NameError: name 'xtrain_bow' is not defined

#### Model evaluation

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score 

def metrics(prediction, actual): 
    print('Confusion_matrix \n', confusion_matrix(actual, prediction))
    print('\nAccuracy:', accuracy_score(actual, prediction))
    print('\nclassification_report\n')
    print(classification_report(actual, prediction))
    
#BOW   
metrics(prediction_bow, ytest)

In [ ]:
#TF-IDF
metrics(prediction_tf, ytest)